## This notebooks solves for: https://brilliant.org/problems/get-a-better-hand-2/

In [1]:
import pandas as pd
import numpy as np
import math

import sys
sys.stdout.flush()

In [2]:
def num_lst(lst):
    return all([isinstance(x, (int, float, complex)) for x in lst])

def f(n):
    return math.factorial(n)

def nCr(n, r):
    return f(n)/(f(r) * f(n-r))

class deck(object):
    def __init__(self, ranks, suits, hand = 5, lowest_rank = 'auto', highest_rank = 'auto'):
        assert num_lst(ranks), 'ranks must be a list of numerics'
        assert isinstance(hand, int), 'hand must be an integer'
        
        self.ranks = ranks
        self.suits = suits
        self.hand = hand
        
        self.num_ranks = len(self.ranks)
        self.num_suits = len(self.suits)
        
        if lowest_rank == 'auto':
            self.lowest_rank = min(self.ranks)
        else:
            self.lowest_rank = lowest_rank
            
        if highest_rank == 'auto':
            self.highest_rank = max(self.ranks)
        else:
            self.highest_rank = highest_rank
            
        self.num_cards = self.num_ranks * self.num_suits
        
        #calculate the number of rank combos for straights:
        fro = self.lowest_rank
        to = self.highest_rank - self.hand + 1
        self.straight_ranks = to - fro + 1       
        
    def all_hands(self):
        return nCr(self.num_cards, self.hand)

    def high_card(self):
        all_ranks = nCr(self.num_ranks, self.hand)
        str_ranks = self.straight_ranks
        all_suits = nCr(self.num_suits, 1) ** self.hand
        kind_suits = self.num_suits
        
        #print('[(', self.num_ranks, ' choose ', self.hand, ') - ', str_ranks, ']', 
        #      '[(', self.num_suits, ' choose ', 1, ') - ', kind_suits, ']')
        
        return (all_ranks - str_ranks) * (all_suits - kind_suits)
    
    def one_pair(self):
        pair_rank = nCr(self.num_ranks, 1)
        pair_suits = nCr(self.num_suits, 2)
        
        num_cards = self.hand - 2
        cards_ranks = nCr(self.num_ranks - 1, num_cards)
        cards_suits = nCr(self.num_suits, 1) ** num_cards
        
        return pair_rank * pair_suits * cards_ranks * cards_suits
    
    def two_pair(self):
        pairs_rank = nCr(self.num_ranks, 2)
        pairs_suits = nCr(self.num_suits, 2) ** 2
        
        num_cards = self.hand - 4
        card_rank = nCr(self.num_ranks - 2, num_cards)
        card_suit = nCr(self.num_suits, num_cards)
        
        return pairs_rank * pairs_suits * card_rank * card_suit
    
    def three_kind(self):
        three_rank = nCr(self.num_ranks, 1)
        three_suits = nCr(self.num_suits, 1)
        
        num_cards = self.hand - 3
        cards_rank = nCr(self.num_ranks - 1, num_cards)
        cards_suits = nCr(self.num_suits, 1) ** num_cards
        
        return three_rank * three_suits * cards_rank * cards_suits
    
    def straight(self):
        straight_ranks = nCr(self.straight_ranks, 1)
        straight_suits = nCr(self.num_suits, 1) ** self.hand
        flush_suit = nCr(self.num_suits, 1)
        
        return straight_ranks * (straight_suits - flush_suit)
    
    def flush(self):
        flush_ranks = nCr(self.num_ranks, self.hand)
        straight_ranks = nCr(self.straight_ranks, 1)
        
        flush_suits = nCr(self.num_suits, 1)
        
        return flush_suits * (flush_ranks - straight_ranks)
    
    def full_house(self):
        three_rank = nCr(self.num_ranks, 1)
        three_suits = nCr(self.num_suits, 3)
        
        pair_rank = nCr(self.num_ranks - 1, 1)
        pair_suits = nCr(self.num_suits, 2)
        
        return three_rank * three_suits * pair_rank * pair_suits
    
    def four_kind(self):
        four_rank = nCr(self.num_ranks, 1)
        four_suits = nCr(self.num_suits, 4)
        
        num_cards = self.hand - 4
        card_rank = nCr(self.num_ranks - 1, 1)
        card_suit = nCr(self.num_suits, 1)
        
        return four_rank * four_suits * card_rank * card_suit
    
    def straight_flush(self):
        straight_ranks = nCr(self.straight_ranks, 1)
        straight_suits = nCr(self.num_suits, 1)
        
        royal_flush = nCr(self.num_suits, 1)
        
        return straight_ranks * straight_suits - royal_flush
    
    def royal_flush(self):
        return nCr(self.num_suits, 1)
        

In [3]:
your_ranks = [2, 3, 4, 5, 6]
your_suits = ['H', 'D', 'S', 'C']

friend_ranks = [7, 8, 9, 10, 11, 12, 13, 14] #Ace is 14
friend_suits = ['H', 'D', 'S', 'C']

In [4]:
whole_deck = deck([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], ['H', 'D', 'S', 'C'], highest_rank = 14)

In [5]:
your_deck = deck([2, 3, 4, 5, 6], ['H', 'D', 'S', 'C'])
friend_deck = deck([7, 8, 9, 10, 11, 12, 13, 14], ['H', 'D', 'S', 'C'])

In [6]:
counts = pd.DataFrame(columns = ['deck', 'high_card', 'one_pair', 'two_pair', 'three_kind', 'straight', 
                                 'flush', 'full_house', 'four_kind', 'straight_flush', 'royal_flush'], 
                      index = ['you', 'your_friend'])

In [7]:
counts.deck = [your_deck, friend_deck]
counts.high_card = counts.deck.apply(lambda x: x.high_card())
counts.one_pair = counts.deck.apply(lambda x: x.one_pair())
counts.two_pair = counts.deck.apply(lambda x: x.two_pair())
counts.three_kind = counts.deck.apply(lambda x: x.three_kind())
counts.straight = counts.deck.apply(lambda x: x.straight())
counts.flush = counts.deck.apply(lambda x: x.flush())
counts.full_house = counts.deck.apply(lambda x: x.full_house())
counts.four_kind = counts.deck.apply(lambda x: x.four_kind())
counts.straight_flush = counts.deck.apply(lambda x: x.straight_flush())
counts.royal_flush = counts.deck.apply(lambda x: x.royal_flush())

counts

,deck,high_card,one_pair,two_pair,three_kind,straight,flush,full_house,four_kind,straight_flush,royal_flush
you,<__main__.deck object at 0x10789a048>,0.0,7680.0,4320.0,1920.0,1020.0,0.0,480.0,80.0,0.0,4.0
your_friend,<__main__.deck object at 0x10789a080>,53040.0,107520.0,24192.0,10752.0,4080.0,208.0,1344.0,224.0,12.0,4.0


In [8]:
probs = counts.drop('deck', axis = 1)
probs.loc['you',] = probs.loc['you',].apply(lambda x: x / your_deck.all_hands())
probs.loc['your_friend',] = probs.loc['your_friend',].apply(lambda x: x / friend_deck.all_hands())
probs

,high_card,one_pair,two_pair,three_kind,straight,flush,full_house,four_kind,straight_flush,royal_flush
you,0.000000,0.495356,0.278638,0.123839,0.065789,0.000000,0.030960,0.005160,0.00000,0.000258
your_friend,0.263388,0.533927,0.120133,0.053393,0.020261,0.001033,0.006674,0.001112,0.00006,0.000020


In [9]:
probs.sum(axis = 1) #check that probability distributions for you and your friend both add up to 1

you            1.0
your_friend    1.0
dtype: float64

In [10]:
your_friend = probs.loc['your_friend',].values
friend_agg = np.cumsum(your_friend)

#if you and your friend have the same kinds of hand, your friend would always win since he/she has higher cards.
#therefore, the probability that you win is equal to the probability that your friend has a lower type of hand then you
#let friend_lower represent the probability that your friend gets a lower type of hand, given the hand that you get

friend_lower = pd.Series(np.append([0], friend_agg[:-1]), name = 'friend_lower', index = probs.columns)
probs = probs.append(friend_lower)
probs

,high_card,one_pair,two_pair,three_kind,straight,flush,full_house,four_kind,straight_flush,royal_flush
you,0.000000,0.495356,0.278638,0.123839,0.065789,0.000000,0.030960,0.005160,0.000000,0.000258
your_friend,0.263388,0.533927,0.120133,0.053393,0.020261,0.001033,0.006674,0.001112,0.000060,0.000020
friend_lower,0.000000,0.263388,0.797314,0.917448,0.970841,0.991101,0.992134,0.998808,0.999921,0.999980


In [11]:
#let you_win represent that probability that you win, given the hand that you get
you_win = pd.Series(np.multiply(probs.loc['you', ], probs.loc['friend_lower', ]), 
                       name = 'you_win', index = probs.columns)
probs = probs.append(you_win)
probs

,high_card,one_pair,two_pair,three_kind,straight,flush,full_house,four_kind,straight_flush,royal_flush
you,0.000000,0.495356,0.278638,0.123839,0.065789,0.000000,0.030960,0.005160,0.000000,0.000258
your_friend,0.263388,0.533927,0.120133,0.053393,0.020261,0.001033,0.006674,0.001112,0.000060,0.000020
friend_lower,0.000000,0.263388,0.797314,0.917448,0.970841,0.991101,0.992134,0.998808,0.999921,0.999980
you_win,0.000000,0.130471,0.222162,0.113616,0.063871,0.000000,0.030716,0.005154,0.000000,0.000258


In [13]:
#p is the probability that you win, over all possible hands
p = probs.loc['you_win',].sum()
#problem asks for the floor of 1000*p:
math.floor(1000*p)

566